In [1]:
import pandas as pd
import regex as re
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk import pos_tag,pos_tag_sents
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split as tts

In [2]:
data= pd.read_csv('D:\\Studies\\DS\\Class\\Naive Bayes\\spam.csv',encoding='latin-1')

In [3]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
df=data[['v2']]

In [5]:
text1=''
for i in df['v2']:
    aa=i.split(' ')
    for a in aa:
        a+=' '
        text1+=a
zumma=text1.split(' ')
fdist1 = nltk.FreqDist(zumma)
print (fdist1.most_common(50))

[('to', 2134), ('you', 1622), ('I', 1466), ('a', 1327), ('the', 1197), ('and', 857), ('in', 798), ('is', 781), ('i', 742), ('u', 692), ('for', 644), ('', 629), ('my', 628), ('of', 591), ('your', 560), ('me', 557), ('on', 487), ('have', 474), ('2', 453), ('that', 421), ('it', 396), ('are', 396), ('call', 376), ('or', 373), ('be', 362), ('at', 359), ('with', 349), ('not', 340), ('will', 330), ('get', 326), ('can', 304), ('U', 299), ('so', 294), ('ur', 293), ("I'm", 286), ('but', 282), ('&lt;#&gt;', 276), ('You', 263), ('from', 256), ('4', 249), ('do', 243), ('up', 241), ('just', 236), ('if', 235), ('.', 235), ('go', 234), ('when', 233), ('know', 224), ('this', 223), ('like', 222)]


In [6]:
input_dat=data.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'])

In [7]:
input_dat['spam']=input_dat['v1'].apply(lambda x: 1 if x=='spam' else 0)

In [8]:
input_dat.head(3)

,v1,v2,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1


In [9]:
input_dat['v2']=input_dat['v2'].str.lower()

In [10]:
input_dat['v2'] = input_dat['v2'].str.replace('http\S+|www.\S+', ' ', case=False)

In [11]:
input_dat['v2'] = input_dat['v2'].str.replace('\d+',' ')

In [12]:
input_dat['v2'] = input_dat['v2'].str.replace('\W', ' ')

In [13]:
input_dat['v2'] = input_dat['v2'].str.replace(' +', ' ')
input_dat['v2'] = input_dat['v2'].str.replace('\s{2,}', ' ')

In [14]:
#Lammatization using textblob
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from textblob import TextBlob, Word

def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n',
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]  
    #dictionary.get(keyname, value)
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

[nltk_data] Downloading package punkt to C:\Users\mathan
[nltk_data]     mohan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mathan mohan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\mathan
[nltk_data]     mohan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
import nltk
nltk.download('wordnet')
input_dat['v2']=input_dat['v2'].apply(lemmatize_with_postag)

[nltk_data] Downloading package wordnet to C:\Users\mathan
[nltk_data]     mohan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
stop_words_nltk=nltk.corpus.stopwords.words('english')

In [17]:
from sklearn.feature_extraction import text
stop_words_sklearn = text.ENGLISH_STOP_WORDS

In [20]:
#Remove the stop words from the text
for word in stop_words_nltk :
    input_dat['v2'] = input_dat['v2'].replace(to_replace=r'\b%s\b'%word, value="",regex=True)

In [23]:
input_dat['v2']=input_dat['v2'].map(lambda x: x.strip())

In [24]:
def pos(text):
    return TextBlob(text).tags

In [25]:
pos_output=input_dat['v2'].apply(pos)

In [26]:
pos_output = pd.DataFrame(pos_output)

In [28]:
input_dat['v2']=pos_output['v2'].map(lambda x:" ".join(["/".join(x) for x in x ]) )

In [29]:
input_train,input_test,outpt_train,output_test=tts(input_dat['v2'],input_dat['spam'],test_size=0.2,random_state=101)

In [30]:
tf_vect = TfidfVectorizer(lowercase = True, use_idf=True, smooth_idf=True,sublinear_tf=False, ngram_range=(2,2))

In [31]:
X_train_tf=tf_vect.fit_transform(input_train)

In [95]:
#tf_vect.get_feature_names()

In [32]:
X_train_tf=pd.DataFrame(X_train_tf.todense(),columns=tf_vect.get_feature_names())

In [33]:
X_test_tf = tf_vect.transform(input_test)

In [34]:
X_test_tf =pd.DataFrame(X_test_tf.todense(),columns=tf_vect.get_feature_names())

In [36]:
X_train_tf.shape

(4457, 19164)

In [37]:
X_test_tf.shape

(1115, 19164)

In [35]:
X_test_tf

,____ nnp,aa nn,aah nn,aaniye vbp,aaooooright jj,aathi nn,abbey vbz,abdomen nns,abeg nns,abel nn,...,û_ nn,û_ nnp,û_ vb,û_thanks nns,ûªm vbp,ûªt nnp,ûò in,ûò jj,ûò nnp,ûò vbd
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

In [101]:
from sklearn.naive_bayes import GaussianNB
model1 = MultinomialNB()
model1.fit(X_train_tf, outpt_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [102]:
pred_Naive=model1.predict(X_test_tf)

In [103]:
metrics.accuracy_score(output_test, pred_Naive)

0.947085201793722

In [104]:
print(classification_report(pred_Naive, output_test))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97      1036
           1       0.57      1.00      0.73        79

    accuracy                           0.95      1115
   macro avg       0.79      0.97      0.85      1115
weighted avg       0.97      0.95      0.95      1115



In [106]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [107]:
model.fit(X_train_tf, outpt_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [108]:
y_predicted = model.predict(X_test_tf)

In [109]:
y_predicted[:10]

array([1, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int64)

In [110]:
model.score(X_test_tf,output_test)

0.9210762331838565

In [111]:
print(classification_report(y_predicted, output_test))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96      1065
           1       0.36      1.00      0.53        50

    accuracy                           0.92      1115
   macro avg       0.68      0.96      0.74      1115
weighted avg       0.97      0.92      0.94      1115

